# QA/QC for delivered administrative boundaries
The World Bank receives regular deliveries of official administrative boundaries. This script will process and evaluate these boundaries in several ways:

- Combine the two ID columns into a single, primary key  
  a. Check to ensure no duplicates in this new, primary key
- Combine ADM0 file with disputed areas shapefile 
- Create ocean mask 
- Within higher heirarchical level, evaluate name duplication
- Perform topological checks  
  a. Ensure no overlaps  
  b. Ensure no gaps  
  c. Ensure all admin1 and admin2 shapes are fully contained within their heirarchical parents
 


In [1]:
import sys, os
import requests
import json

import geopandas as gpd
import pandas as pd

from shapely.geometry import Point, Polygon, box, shape
from GOSTrocks.misc import tPrint

sys.path.append("../src")

from wb_gad_helper import *

%load_ext autoreload
%autoreload 2  

In [2]:
admin_folder = r'C:\WBG\Work\data\ADMIN\NEW_WB_BOUNDS'
out_folder = r'C:\WBG\Work\data\ADMIN\QAQC'
final_folder = os.path.join(admin_folder, 'FOR_PUBLICATION')
if not os.path.exists(final_folder):
    os.makedirs(final_folder)
for file_format in ['gpkg', 'shp', 'geojson']:
    temp_folder = os.path.join(final_folder, file_format)
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)

if not os.path.exists(out_folder):
    os.makedirs(out_folder)

admin0_file = os.path.join(admin_folder, 'WB_GAD_ADM0.shp')
adm0_disputes_file = os.path.join(admin_folder, 'WB_GAD_NDLSA.shp')
admin1_file = os.path.join(admin_folder, 'WB_GAD_ADM1.shp')
admin2_file = os.path.join(admin_folder, 'WB_GAD_ADM2.shp')

In [3]:
adm0 = gpd.read_file(admin0_file)
adm0_disputes = gpd.read_file(adm0_disputes_file)
adm1 = gpd.read_file(admin1_file)
adm2 = gpd.read_file(admin2_file)

In [4]:
# Add World Bank classifications to ADM0
wb_classes = get_wb_classifications_strict(grouping_version="37.0", region_version="2.0", income_version="2.0",)
wb_classes.rename({'ISO3':'ISO_A3', 'REGION':'WB_REGION', 'INCOME':'WB_INCOME'}, inplace=True, axis=1)
wb_classes['CONTINENT'] = wb_classes['CONTINENT'].fillna("001")
continent_names_map = {
    '002':'Africa',
    '005':'South America',
    '009':'Oceania',
    '142':'Asia',
    '150':'Europe',
    '001':'North America'
}
wb_classes['CONTINENT'] = wb_classes['CONTINENT'].map(continent_names_map)

adm0.drop("WB_REGION", axis=1, inplace=True)
adm0 = pd.merge(adm0, wb_classes, on='ISO_A3')
adm0.head()

,ISO_A3,ISO_A2,WB_A3,HASC_0,GAUL_0,WB_STATUS,SOVEREIGN,NAM_0,geometry,CONTINENT,WB_REGION,WB_INCOME
0,CHN,CN,CHN,CN,147295,Member State,CHN,China,"MULTIPOLYGON (((117.58675 38.59517, 117.58909 ...",Asia,EAS,UMC
1,JPN,JP,JPN,JP,126,Member State,JPN,Japan,"MULTIPOLYGON (((137.48411 34.67386, 137.46683 ...",Asia,EAS,HIC
2,KOR,KR,KOR,KR,202,Member State,KOR,Republic of Korea,"MULTIPOLYGON (((126.05363 36.19852, 126.05372 ...",Asia,EAS,HIC
3,PRK,KP,PRK,KP,67,Non Member State,PRK,D. P. R. of Korea,"MULTIPOLYGON (((126.95508 38.16282, 126.95184 ...",Asia,EAS,LIC
4,RUS,RU,RUS,RU,204,Member State,RUS,Russian Federation,"MULTIPOLYGON (((130.61904 48.88019, 130.60659 ...",Europe,ECS,UMC


## Combine ID columns

In [5]:
merged_adm1 = merge_id_columns(adm1, [['P_CODE_1', 'P_CODE_1_t'], ['ADM1CD', 'ADM1CD_t']])
merged_adm2 = merge_id_columns(adm2, [['P_CODE_1', 'P_CODE_1_t'], ['P_CODE_2', 'P_CODE_2_t'], ['ADM1CD', 'ADM1CD_t'], ['ADM2CD', 'ADM2CD_t']])

In [6]:
# Check for duplicates in ADM1
test_col = 'ADM1CD_c'
check_duplicates(merged_adm1, test_col, os.path.join(out_folder, f'adm1_duplicates_{test_col}.gpkg'))

# Check for duplicates in ADM2
test_col = 'ADM2CD_c'
check_duplicates(merged_adm2, test_col, os.path.join(out_folder, f'adm2_duplicates_{test_col}.gpkg'))

ADM1CD_c duplicates: 0
ADM2CD_c duplicates: 0


## Combine ADM0 with disputed territories

In [7]:
for col in adm0_disputes.columns:
    if not col in adm0.columns:
        adm0_disputes.drop(columns=[col], inplace=True)
adm0_disputes.head()
adm0_complete = pd.concat([adm0, adm0_disputes], ignore_index=True)

In [8]:
# Generate a global ocean mask from the admin divisions
ocean_polygon = box(-180, -90, 180, 90)  # Global bounding box for ocean
#clip ocean polygon to adm0 boundaries
ocean_polygon = ocean_polygon.difference(adm0_complete.union_all())  # Remove land areas
ocean_mask = gpd.GeoDataFrame([[1, ocean_polygon]], columns=['id', 'geometry'], crs=4326)


## Evaluate name duplication

In [9]:
# Evaluate duplicate names in adm1 and adm2
evaluate_duplicate_names(merged_adm1, 'NAM_1', 'ISO_A3', os.path.join(out_folder, "ADM1_name_duplicates.log"))
# Evaluate duplicate names in adm1 and adm2
evaluate_duplicate_names(merged_adm2, 'NAM_2', 'ADM1CD_c', os.path.join(out_folder, "ADM2_name_duplicates.log"))

In [10]:
# Use sjoin to identify overlapping polygons
'''
sj = gpd.sjoin(adm0, adm0, how="inner", predicate="overlaps", lsuffix="left", rsuffix="right")
sj = sj[sj.index != sj.index_right]

sj['intersection_geom'] = sj['geometry_left'].intersection(sj['geometry_right'])
sj['intersection_area'] = sj['intersection_geom'].area
sj
'''

'\nsj = gpd.sjoin(adm0, adm0, how="inner", predicate="overlaps", lsuffix="left", rsuffix="right")\nsj = sj[sj.index != sj.index_right]\n\nsj[\'intersection_geom\'] = sj[\'geometry_left\'].intersection(sj[\'geometry_right\'])\nsj[\'intersection_area\'] = sj[\'intersection_geom\'].area\nsj\n'

## Separate datasets into primary and secondary tables

The delivered files contain several columns that are temporary or reference external sources. This section will separate the superfluous columns into a secondary table

In [11]:
adm1_primary = 'ADM1CD_c'
adm1_simple_cols = ['ISO_A3','ISO_A2','WB_A3','WB_REGION','WB_STATUS','NAM_0','NAM_1','ADM1CD_c','GEOM_SRCE', 'geometry']
adm1_bad_cols = [adm1_primary] + [x for x in merged_adm1.columns if x not in adm1_simple_cols]

adm2_primary = 'ADM2CD_c'
adm2_simple_cols = adm1_simple_cols + ['NAM_2','ADM2CD_c']
adm2_bad_cols = [adm2_primary] + [x for x in merged_adm2.columns if x not in adm2_simple_cols]

# Write out final versions in several data formats

write out:
- adm0 base
- adm0 NDLSA
- adm0 complete
- adm1 simple
- adm1 supplemntal columns
- adm2 simple 
- adm2 supplemental columns

in these formats
- geopackage
- geojson
- shapefile

In [ ]:
for file_def in [
    (ocean_mask, 'WB_GAD_ocean_mask'),
    (adm0_complete, 'WB_GAD_ADM0_complete'),
    (adm0, 'WB_GAD_ADM0'),
    (adm0_disputes, 'WB_GAD_ADM0_NDLSA'),
    (merged_adm1.loc[:, adm1_simple_cols], 'WB_GAD_ADM1'),
    (merged_adm2.loc[:, adm2_simple_cols], 'WB_GAD_ADM2'),    
    ]:
    gdf, filename = file_def
    write_output(gdf, final_folder, filename)
    tPrint(f"completed {filename}")
# Save additional columns to CSV for reference
pd.DataFrame(merged_adm1.loc[:, adm1_bad_cols]).to_csv(os.path.join(final_folder, 'WB_GAD_adm1_additional_columns.csv'), index=False)
pd.DataFrame(merged_adm2.loc[:, adm2_bad_cols]).to_csv(os.path.join(final_folder, 'WB_GAD_adm2_additional_columns.csv'), index=False)

# Generate dissolved continents and regions
dissolve_folder = os.path.join(final_folder, 'REGIONS')
if not os.path.exists(dissolve_folder):
    os.makedirs(dissolve_folder)
for col in ['CONTINENT', 'WB_REGION']:
    if col in adm0_complete.columns:
        tPrint(f"Dissolving by {col}")
        dissolved = adm0_complete.dissolve(by=col, as_index=False)
        # project results to equal-area projection
        dissolved = dissolved.to_crs(epsg=8857)
        dissolved.to_file(os.path.join(dissolve_folder, f"WB_GAD_{col}.gpkg"), driver='GPKG')
tPrint("All output complete")

14:21:50	completed WB_GAD_ocean_mask
14:29:53	completed WB_GAD_ADM0_complete
14:38:26	completed WB_GAD_ADM0
14:38:28	completed WB_GAD_ADM0_NDLSA
14:43:47	completed WB_GAD_ADM1
15:06:02	completed WB_GAD_ADM2
15:06:04	Dissolving by CONTINENT
15:08:43	Dissolving by WB_REGION
